In [1]:
#!pip install rank-bm25
#from rank_bm25 import BM25Okapi
import pandas as pd
#import matplotlib.pyplot as plt
#import seaborn as sns
import os
from os import listdir
from os.path import isfile, join
import re
import numpy as np
from math import floor, ceil

import json
import gzip

from os import walk
from scipy.spatial import KDTree

# !pip install geopy
# !pip install phonenumbers
# !pip install pycountry

import geopy.distance
import phonenumbers
import pycountry



In [2]:
pd.options.display.max_columns = 100

In [3]:
path = r"../../../../src/data"
lb_path_min3 = path + r"/LocalBusiness/LocalBusiness_minimum3/geo_preprocessed"
lb_path_top100 = path + r"/LocalBusiness/LocalBusiness_top100/geo_preprocessed"
lb_path_rest = path + r"/LocalBusiness/LocalBusiness_rest/geo_preprocessed"
rest_path_min3 = path + r"/Restaurant/Restaurant_minimum3/geo_preprocessed"
rest_path_top100 = path + r"/Restaurant/Restaurant_top100/geo_preprocessed"
rest_path_rest = path + r"/Restaurant/Restaurant_rest/geo_preprocessed"
hotel_path_min3 = path + r"/Hotel/Hotel_minimum3/geo_preprocessed"
hotel_path_top100 = path + r"/Hotel/Hotel_top100/geo_preprocessed"
hotel_path_rest = path + r"/Hotel/Hotel_rest/geo_preprocessed"


file_path_list = [lb_path_min3, lb_path_top100, rest_path_min3, rest_path_top100, hotel_path_min3, hotel_path_top100, lb_path_rest, rest_path_rest, hotel_path_rest ]

In [4]:
def create_df(file_path):
    files = os.listdir(file_path)
    df_as_list = []
    for lb in files:
        with gzip.open(file_path + '/' + lb, 'r') as dataFile:
            for line in dataFile:
                lineData = json.loads(line.decode('utf-8'))
                lineData["origin"] = lb
                df_as_list.append(lineData)
    df = pd.DataFrame(df_as_list)
    return df

In [5]:
df_list = []
for file_path in file_path_list:
    df = create_df(file_path)
    df_list.append(df)

## Concatenate Dataframes

In [6]:
df_all = pd.concat(df_list, axis = 0, ignore_index = True)

In [7]:
len(df_all)

4856255

In [32]:
print(df_all['origin'].nunique())

295428


In [33]:
df_all.tail()

,row_id,name,address,description,pricerange,founder,owns,page_url,addressregion,streetaddress,addresscountry,telephone_address,addresslocality,postalcode,origin,telephone,geo,sameas,latitude,longitude,url,aggregaterating,image,openinghours,review,email,contactpoint,reviews,employees,faxnumber,about,citystatezip,openinghoursspecification,areaserved,location,paymentaccepted,currenciesaccepted,legalname,photos,photo,logo,alternatename,brand,department,makesoffer,map,award,hasmap,isicv4,foundingdate,...,sms,mailto,tollffee,dissolutiondate,adressregion,worksfor,opening_hours,email_address,slug,"\u201caddressregion\""",line,isfamilyfriendly,actors,faxno,addresslocalilty,subtitle,t\u00e9l\u00e9phone,weekendhours,secondarytelephone,primarytelephone,weekdayhours,addressinfo,menusection,deliverymethod,streetaddressline2,hasmenusection,telepone,cccc,hasdescription,resources,facilities-list,vatnumber,streetaddressline1,numadults,mobilenumber,@image,@logo,result,google+,numchildren,local,floorlevel,yearbuilt,value,itemlistorder,typeofgood,hotelname,piva,offersservice,hasamenity
4856250,0,Majycc,"{'addresslocality': 'Paris', 'addresscountry':...",Majycc,NaN,NaN,NaN,https://www.majycc.com/fonciere,NaN,10 Rue Treilhard,FR,NaN,Paris,75008,Hotel_majycc.com_September2020.json.gz,XX XX XX XX XX,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4856251,0,Soggiorno La Coccinella - Florence - Italy,"{'postalcode': '50144', 'addresslocality': 'Fl...",Soggiorno La Coccinella is located in the Port...,NaN,NaN,NaN,https://www.soggiornolacoccinella.com/en-us,NaN,Viale Belfiore 58,NaN,NaN,Florence,50144,Hotel_soggiornolacoccinella.com_September2020....,+39 392 220 7801,NaN,NaN,NaN,NaN,NaN,"{'ratingvalue': '8.1E0', 'reviewcount': '265',...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4856252,0,Regency Inn & Suites Florida,"{'addresslocality': 'De Funiak Springs', 'addr...",Regency Inn &amp; Suites is an affordable fami...,NaN,NaN,NaN,https://www.regencyinnfl.com/,Florida,472 Hugh Adams Rd,USA,NaN,De Funiak Springs,32435,Hotel_regencyinnfl.com_September2020.json.gz,(850) 892-6115,"{'latitude': '30.694060', 'longitude': '-86.12...",NaN,30.694060,-86.122010,NaN,"{'ratingvalue': '5', 'bestrating': '5', 'revie...",NaN,NaN,NaN,Info@regencyinndefuniak.com,"{'contacttype': 'Customer Support', 'telephone...",NaN,NaN,(850) 892-0707,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4856253,0,Samui Green Hotel,"{'addresslocality': 'Koh Samui', 'addressregio...",NaN,NaN,NaN,NaN,https://samuigreenhotel.com/,Surat Thani,"17/52 Moo 3 Chaweng Beach Road, Amphoe",Thailand,NaN,Koh Samui,84320,Hotel_samuigreenhotel.com_September2020.json.gz,+66-(0)77-422-590,"{'longitude': '100.0576° E', 'latitude': '9.52...",NaN,9.5216° N,100.0576° E,NaN,NaN,NaN,"Monday,Tuesday,Wednesday,Thursday,Friday,Satur...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4856254,0,Fawlty Towers,"{'streetaddress': 'Asheldon Rd', 'addressregio...","Lorem ipsum dolor si

# Tables after getting rid off empy phone - and country codes

In [34]:
df_clean_phone = df_all[df_all["addresscountry"].notna()]
df_clean_phone = df_clean_phone[df_clean_phone["telephone"].notna()]
#len(df_clean_phone)
df_clean_phone["origin"].nunique()

165207

# Tables after getting rid empy phone - and country codes

In [11]:
# df_clean_geo = df_all[df_all["longitude"].notna()]
# df_clean_geo = df_clean_geo[df_clean_geo["latitude"].notna()]
# df_clean_geo["origin"].nunique()

In [12]:
# df_clean_phone_geo = df_clean_phone[df_clean_phone["longitude"].notna()]
# df_clean_phone_geo = df_clean_phone_geo[df_clean_phone_geo["latitude"].notna()]
# df_clean_phone_geo["origin"].nunique()

### Roughly 1.3 mio data after conditioning on phone and non-zero country codes

## Format longitudes AND latitudes

In [13]:
# lon = "longitude"
# lat = "latitude"

In [14]:
# # Remove entries that are not numbers or cannot be convertred to one number (list etc.)
# longitudes = df_clean_geo[lon].to_numpy()
# latitudes = df_clean_geo[lat].to_numpy()
# deleteList = []
# i = 0

# for value in longitudes:
#     if ((isinstance(value, str) == False) & (isinstance(value, float) == False)):
#         deleteList.append(i)
#     i = i + 1

# i = 0
# for value in latitudes:
#     if ((isinstance(value, str) == False) & (isinstance(value, float) == False)):
#         deleteList.append(i)
#     i = i + 1    

# df_clean_geo.drop(df_clean_geo.index[deleteList], axis = 0, inplace = True)        

### Format longitude and latitude

In [15]:
# longArray = df_clean_geo[lon].to_numpy().astype(str)
# longArray = np.char.replace(longArray, ',', '.')
# longArray = np.char.replace(longArray, '--', '-')
# df_clean_geo[lon] = longArray
# df_clean_geo[lon] = pd.to_numeric(df_clean_geo[lon], errors='coerce')

# latArray = df_clean_geo[lat].to_numpy().astype(str)
# latArray = np.char.replace(latArray, ',', '.')
# latArray = np.char.replace(latArray, '--', '-')
# df_clean_geo[lat] = latArray
# df_clean_geo[lat] = pd.to_numeric(df_clean_geo[lat], errors='coerce')

In [16]:
# # Remove the entries that were set to NaN because of other errors
# df_clean_geo = df_clean_geo[df_clean_geo["longitude"].notna()]
# df_clean_geo = df_clean_geo[df_clean_geo["latitude"].notna()]
# # Make sure to only include valid longitudes and latitudes
# df_clean_geo = df_clean_geo.loc[(df_clean_geo[lat] >= -90) & (df_clean_geo[lat] <= 90)]
# df_clean_geo = df_clean_geo.loc[(df_clean_geo[lon] >= -180) & (df_clean_geo[lon] <= 80)]

In [17]:
# df_clean_geo["origin"].nunqiue()

### Roughly 700k datapoints left after conditioning on geo-location

## Further preprocessing step

### Remove non-digits from telephone numbers

In [18]:
def remove_non_digits(string):
    cleaned = re.sub('[^0-9]','', string)
    return cleaned

In [19]:
df_clean = df_clean_phone[["row_id", "address", "page_url", "origin", "telephone" , "addresscountry", "longitude", "latitude"]]

In [20]:
df_clean['telephone_'] = df_clean['telephone'].astype('str').apply(remove_non_digits)

<ipython-input-20-cbc6e0b80647>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_clean['telephone_'] = df_clean['telephone'].astype('str').apply(remove_non_digits)


### Extract country codes to ISO-2 format using ``pycountry``

In [21]:
countries = {}
for country in pycountry.countries:
    countries[country.name] = country.alpha_2

countries


{'Aruba': 'AW',
 'Afghanistan': 'AF',
 'Angola': 'AO',
 'Anguilla': 'AI',
 'Åland Islands': 'AX',
 'Albania': 'AL',
 'Andorra': 'AD',
 'United Arab Emirates': 'AE',
 'Argentina': 'AR',
 'Armenia': 'AM',
 'American Samoa': 'AS',
 'Antarctica': 'AQ',
 'French Southern Territories': 'TF',
 'Antigua and Barbuda': 'AG',
 'Australia': 'AU',
 'Austria': 'AT',
 'Azerbaijan': 'AZ',
 'Burundi': 'BI',
 'Belgium': 'BE',
 'Benin': 'BJ',
 'Bonaire, Sint Eustatius and Saba': 'BQ',
 'Burkina Faso': 'BF',
 'Bangladesh': 'BD',
 'Bulgaria': 'BG',
 'Bahrain': 'BH',
 'Bahamas': 'BS',
 'Bosnia and Herzegovina': 'BA',
 'Saint Barthélemy': 'BL',
 'Belarus': 'BY',
 'Belize': 'BZ',
 'Bermuda': 'BM',
 'Bolivia, Plurinational State of': 'BO',
 'Brazil': 'BR',
 'Barbados': 'BB',
 'Brunei Darussalam': 'BN',
 'Bhutan': 'BT',
 'Bouvet Island': 'BV',
 'Botswana': 'BW',
 'Central African Republic': 'CF',
 'Canada': 'CA',
 'Cocos (Keeling) Islands': 'CC',
 'Switzerland': 'CH',
 'Chile': 'CL',
 'China': 'CN',
 "Côte d'Iv

In [22]:
# fuction to modify the country dictionary in uppercase
def modify_dic(d):
    for key in list(d.keys()):
        new_key = key.upper()
        d[new_key] = d[key]
    return d

In [23]:
countries_upper = modify_dic(countries)

In [24]:
#countries_upper

In [25]:
#uppercase the df_column 
df_clean["addresscountry"] = df_clean["addresscountry"].str.upper()

<ipython-input-25-f93ebcb01688>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_clean["addresscountry"] = df_clean["addresscountry"].str.upper()


In [26]:
df_clean["addresscountry"].value_counts()

US                                         321793
UNITED STATES                              259972
USA                                         88901
UNITED KINGDOM                              61260
NL                                          60967
                                            ...  
BOLIVYA                                         1
BERKSHIRE                                       1
SUNDANCE CENTER                                 1
COLUMBUS STATES                                 1
3-4 COVENTRY STREET, LONDON W1D 6BL, UK         1
Name: addresscountry, Length: 2648, dtype: int64

In [27]:
# Replace known countries with ISO-2 format country code
for key, value in countries_upper.items():
    df_clean["addresscountry"] = df_clean["addresscountry"].str.replace(key, value)

<ipython-input-27-05d220ee92af>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_clean["addresscountry"] = df_clean["addresscountry"].str.replace(key, value)
<ipython-input-27-05d220ee92af>:3: FutureWarning: The default value of regex will change from True to False in a future version.
  df_clean["addresscountry"] = df_clean["addresscountry"].str.replace(key, value)


## Manually normalize countries which do not exist in country package

In [28]:
df_clean["addresscountry"].value_counts().head(30)

US         581765
GB         109552
USA         88901
AU          86579
IN          66515
NL          62617
FR          48612
CA          35543
UK          31874
DE          22687
BR          20344
MX          17592
ES          17468
IT          14254
JP          11273
ID          10662
CL           8507
US,          7959
AE           6406
DK           6244
TR           6000
TH           4765
PT           4363
MY           4301
ENGLAND      4138
AT           4043
CO           4043
CN           3960
SG           3933
AR           3560
Name: addresscountry, dtype: int64

In [29]:
country_dictionary = {
                      "UNITED STATES": "US",
                      "USA":"US",
                      "UNITED KINGDOM": "GB",
                      "UK": "GB",
                      "CANADA": "CA",
                      "AUSTRALIA": "AU",
                      "UNITED ARAB EMIRATES":"AE",
                        "UAE": "AE",
                      "INDIA" : "IN",
                      "NEW ZEALAND": "NZ",
                      "SVERIGE" : "SE",
                      "DEUTSCHLAND": "DE",
                        "DEU": "DE",
                      "RUSSIA": "RU",
                      "ITALIA": "IT",
                      "IRAN": "IR",
                      ", IN" : "IN",
                      "ENGLAND": "GB",
                        "FRA": "FR"
                    }
for key, value in country_dictionary.items():
    df_clean["addresscountry"] = df_clean["addresscountry"].str.replace(key, value)

<ipython-input-29-74035b11ab3c>:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_clean["addresscountry"] = df_clean["addresscountry"].str.replace(key, value)


## In this manual step we save about 50k extra datapoints

## Remove non-covered countries

### There are still some uncovered cases left which have to be removed

In [30]:
df_clean.reset_index(inplace=True)

In [31]:
liste = []
for i, row in enumerate(df_clean["addresscountry"]):
    if len(row) > 2:
        liste.append(i)
        
df_clean = df_clean.drop(liste)        

TypeError: object of type 'float' has no len()

In [ ]:
df_clean["addresscountry"].unique()

## Drop empty phonenumbers and too lengthy phone numbers

In [35]:
df_clean = df_clean[df_clean["telephone_"] != "" ]

In [36]:
liste = []
df_clean.reset_index(inplace=True)
for row_index in df_clean.index:
    if len(df_clean.iloc[row_index]["telephone_"])>18:
        liste.append(row_index)

In [37]:
df_clean.drop(labels = liste, inplace = True)

In [38]:
df_clean = df_clean.drop(columns = ["level_0","index"])
df_clean.tail()

,row_id,address,page_url,origin,telephone,addresscountry,longitude,latitude,telephone_
1423043,0,"{'postalcode': '29577', 'streetaddress': '803 ...",https://www.seacrestmyrtlebeachresort.com/blog...,Hotel_seacrestmyrtlebeachresort.com_September2...,1-800-845-1112,AMERICA,NaN,NaN,18008451112
1423044,0,"{'streetaddress': '1433 Conway Road', 'address...",https://www.fairwaysuitespeak.com/blog/winter-...,Hotel_fairwaysuitespeak.com_September2020.json.gz,(716) 355-5014,US,-79.7276271,42.062021,7163555014
1423045,0,"{'addresslocality': 'Boston', 'addresscountry'...",https://www.eliothotel.com/,Hotel_eliothotel.com_September2020.json.gz,(617) 267-1607,US,NaN,NaN,6172671607
1423046,0,"{'addresslocality': 'De Funiak Springs', 'addr...",https://www.regencyinnfl.com/,Hotel_regencyinnfl.com_September2020.json.gz,(850) 892-6115,US,-86.122010,30.694060,8508926115
1423047,0,"{'addresslocality': 'Koh Samui', 'addressregio...",https://samuigreenhotel.com/,Hotel_samuigreenhotel.com_September2020.json.gz,+66-(0)77-422-590,TH,100.0576° E,9.5216° N,66077422590


In [ ]:
len(df_clean)

## Define normalizer for telephone package phonenumbers

In [39]:
def normalizer(entity):
    number = entity["telephone_"]
    address_country = entity["addresscountry"]
    phone_number = phonenumbers.parse(number, address_country)
    return phone_number

## Finally normalizing phone numbers in E.164 format

### Ignore those which can not be identified and replace as ``unknown``

In [40]:
df_clean.reset_index(inplace=True)
phone_objects =[]
#index = []
for row_index in df_clean.index:
    try:
        phone_object =  normalizer(df_clean.iloc[row_index])
        #index.append(row_index)
        phone_objects.append(phone_object)
    except:
        phone_objects.append("unknown")
    
    

In [ ]:
len(phone_objects)

In [ ]:
df_clean["phone_object"] = pd.Series(phone_objects)

In [ ]:
df_clean = df_clean.drop(columns = "index")
df_clean.head()

In [ ]:
unknown_rows = df_clean[df_clean["phone_object"] == "unknown"].index

In [ ]:
df_clean = df_clean.drop(unknown_rows)

In [ ]:
len(df_clean)

## Check whether phonenumbers are valid

In [ ]:
df_valid_numbers = df_clean[df_clean["phone_object"].apply(phonenumbers.is_valid_number)]

In [ ]:
len(df_valid_numbers)

## Next step: Format every telephone number into unique E.164 format

In [ ]:
#phonenumbers.format_number(df_valid_numbers["phone_object"][0], phonenumbers.PhoneNumberFormat.E164)

In [ ]:
df_valid_numbers["E.164 format"] = df_valid_numbers["phone_object"].apply(lambda objects: phonenumbers.format_number(objects, phonenumbers.PhoneNumberFormat.E164))

In [ ]:
len(df_valid_numbers)

In [ ]:
df_valid_numbers.head()

## After formatting phone numbers into unified format we can group by phone numbers to identify clusters

In [ ]:
df_valid_numbers["E.164 format"].value_counts().sort_values().tail(20)

## Adding the matching telephone numbers in a new column

In [ ]:
df_valid_numbers['telephoneNorm'] = df_valid_numbers['E.164 format'].str.replace('+','').astype(np.int64)

In [ ]:
# def createKDTree(tupleArray):
#     tree = KDTree(tupleArray)
#     return tree

# # Return all values that are in a specific proximity
# def queryTree(tree, point, r = 0):    
#     point = [float(i) for i in point]
#     idx = tree.query_ball_point(point, r)
#     return idx


# df_valid_numbers.reset_index(drop=True, inplace=True)
# df_valid_numbers['indexValue'] = df_valid_numbers.index
# telephoneArray = df_valid_numbers['telephoneNorm'].to_numpy().astype('int64')
# fillArray = np.full(len(telephoneArray), 1)
# tupleArray = np.array((telephoneArray, fillArray)).T.astype('int64')


# # create new column with all matching points
# tree = createKDTree(tupleArray)
# idx = queryTree(tree, tupleArray[0])

# # Search for the closest neighbour in all of the points
# df_valid_numbers['MatchingNumbers'] = df_valid_numbers.apply(lambda row: queryTree(tree,[row['telephoneNorm'], 1]), axis=1) 

In [ ]:
# len(df_valid_numbers)

In [ ]:
# # # filter out the values which only have one value
# data = df_valid_numbers[df_valid_numbers['MatchingNumbers'].apply(lambda x: len(x) > 1)]

## Throwing out noisy clusters

### Throw out everything > 100 (interim step)
### Some peaks come fromt the same table --> could delete those entries right away

In [ ]:
# data["telephoneNorm"].value_counts()

In [ ]:
# data.loc[df_valid_numbers["telephoneNorm"] == 18884082399]

In [ ]:
# len(data)

In [ ]:
# data.head()

## Additional Filtering by Geo Location

In [ ]:
# def calcDifference(pointOne, pointTwo):
#     return geopy.distance.great_circle(pointOne, pointTwo).km

# def calcDifferenceFromRow (row):
#     tmp = data
#     indexValue = row['indexValue']
#     indexPosition = (row[lat], row[lon])
#     diffList = []
#     for value in row['MatchingGeoPoints']:
#         if not value in tmp.index:
#             continue
#         currRow = data.loc[data['indexValue'] == value]
#         currIndex = currRow['indexValue'].values[0]
#         if currIndex == indexValue:
#             diffList.append(-1)
#         else:            
#             currPosition = (currRow[lat].values[0], currRow[lon].values[0])
#             diffList.append(calcDifference(indexPosition, currPosition))
#     return diffList


## Experiment with different radius --> check radius = 0.001 ?

In [ ]:
# def createKDTree(tupleArray):
#     tree = KDTree(tupleArray)
#     return tree

# # Return all values that are in a specific proximity
# def queryTree(tree, point, radius = 0.001):    
#     point = [float(i) for i in point]
#     idx = tree.query_ball_point(point, r=radius)
#     return idx
#     idx = tree.query(point, k=neighbours)
#     return idx[1]

# # convert to tuples and from string to float
# lonArr = data[lon].to_numpy()
# latArr = data[lat].to_numpy()
# tupleArray = np.array((lonArr, latArr)).T.astype('float32')

# data.reset_index(drop=True, inplace=True)
# data['indexValue'] = data.index

# # create new column with all matching points
# tree = createKDTree(tupleArray)
# idx = queryTree(tree, tupleArray[0])

# # Search for the closest neighbour in all of the points
# data['MatchingGeoPoints'] = data.apply(lambda row: queryTree(tree,[row[lon], row[lat]]), axis=1) 

# # Keep those that have one or more matches withing the radius
# data = data[data['MatchingGeoPoints'].apply(lambda x: len(x) > 1)]

In [ ]:
# data["telephoneNorm"].value_counts().tail(20)

In [ ]:
# pd.set_option('max_colwidth', 400)

In [ ]:
# data.loc[data["telephoneNorm"] == 16103991390]

## !Export table with condition and without condition !

In [ ]:
# data.loc[data['indexValue'] == 5]

In [ ]:
# Calculate the difference in km between those
#data['Difference'] = data.apply(lambda row: calcDifferenceFromRow(row), axis=1) 

In [ ]:
# data.iloc[3:4]

In [ ]:
# len(data)

In [ ]:
# data["origin"].value_counts()

In [ ]:
# data.loc[data['indexValue'] == 32][['name', 'address', 'page_url', 'E.164 format', lat, lon]]

In [ ]:
# data.loc[data['indexValue'] == 21907]

In [ ]:
df_valid_numbers.to_json("Concatenated_MatchingFile_with_rest", compression="gzip", orient='records', lines=True)